<a href="https://colab.research.google.com/github/nscWATANABELab/RC_Tank/blob/main/Python_Model_Python_Class_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 必要なライブラリをインポートします

In [1]:
# In[1]:
from datetime import datetime
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout


# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

logdir = "=[=" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

# データセットを作る

In [ ]:
DATADIR = "drive/MyDrive/tank-data/"
CATEGORIES = ["0","1","2","3"]
IMG_SIZE = 100
training_data = []
# In[4]:
def create_training_data():
    for class_num,category in enumerate(CATEGORIES):
        path = os.path.join(DATADIR,category)
        for image_name in os.listdir(path):
            try:
                img_array =cv2.imread(os.path.join(path,image_name),)
                img_resize_array = cv2.resize(img_array,(200,200))
                training_data.append([img_resize_array,class_num])
            except Exception as e:
                print('error')
                pass
create_training_data()

X = []
y = []
for feature,label in training_data:
    X.append(feature)
    y.append(label)
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(y_train)

#学習する

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from tensorflow.keras.applications.inception_v3 import InceptionV3

def get_model():
    base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(200, 200, 3))
    for layers in base_model.layers:
        layers.trainable = False

    # model = Flatten()(base_model.output)
    # model = Dense(128, activation='relu')(model)
    # model = Dropout(0.5)(model)
    # model = Dense(5, activation='softmax')(model)

    model = Flatten()(base_model.output)
    model = Dense(128, activation='relu')(model)
    model = Dropout(0.5)(model)
    model = Dense(4, activation='softmax')(model)

    model = Model(inputs=base_model.input, outputs=model)
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     model.fit(X_train, y_train, batch_size=128, epochs=50)

    return model


# In[ ]:


model = get_model()

# In[ ]:


y_test = np.eye(4)[y]
y_train = np.eye(4)[y_train]


# In[ ]:


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=100,validation_split=0.5, epochs=10,callbacks=[tensorboard_callback])
model.save('model-wataver1.h5')

